# Veri Hazırlama

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

data = pd.read_csv("clean_ner_data.csv").fillna(method="ffill")
data.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
Sentence #,Sentence: 0,Sentence: 0,Sentence: 0,Sentence: 0,Sentence: 0
Word,Corina,Casanova,",",İsviçre,Federal
Tag,B-PERSON,I-PERSON,O,B-LOCATION,O


In [ ]:
class GetSentence(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = GetSentence(data)

In [ ]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
sentences[0]

['Corina', 'Casanova', ',', 'İsviçre', 'Federal', 'Şansölyesidir', '.']

In [ ]:
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
labels[0]

['B-PERSON', 'I-PERSON', 'O', 'B-LOCATION', 'O', 'B-MISC', 'O']

In [ ]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

tag2idx

{'B-LOCATION': 1,
 'B-MISC': 0,
 'B-ORGANIZATION': 3,
 'B-PERSON': 4,
 'I-LOCATION': 8,
 'I-MISC': 5,
 'I-ORGANIZATION': 6,
 'I-PERSON': 7,
 'O': 2,
 'PAD': 9}

# Modelleme

In [ ]:
#Gerekli kütüphaneleri yüklüyoruz.
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

torch.__version__

'1.5.0+cu101'

In [ ]:
# Girdi uzunluğunu maksimum 75 olarak belirtiyoruz.
MAX_LEN = 75

In [ ]:
# GPU’yu kontrol ediyoruz ve onu modelde kullanmak için device değişkenine tanımlıyoruz.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
# Önceden eğitilmiş olan Bert'i tokenizer olarak tanımlıyoruz.
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased', do_lower_case=False)

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Kelimeyi tokenize ediyoruz.Cümle, kelime, harf veya n-gramlar halinde parçalama işlemine tokenization deniyor.
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Tokenize edilmiş sözcüğü son tokenize edilmiş sözcük listesine ekliyoruz.
        tokenized_sentence.extend(tokenized_word)

        # Etiketleri listeye ekliyoruz.
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
input_ids = keras.preprocessing.sequence.pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN,
                          dtype="long",
                          value=0.0,
                          truncating="post",
                          padding="post")

In [ ]:
tags = keras.preprocessing.sequence.pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, 
                     value=tag2idx["PAD"],
                     padding="post",
                     dtype="long", 
                     truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, 
                                                            tags,
                                                            random_state=0,
                                                            test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks,
                                             input_ids,
                                             random_state=0,
                                             test_size=0.1)

In [ ]:
# input,tags ve maskleri Pytorch yapısına uygun hale getiriyoruz.

tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
# Datamızı da aynı şekilde Pytorch yapısına uygun hale getiriyoruz.
train_data = TensorDataset(tr_inputs,
                           tr_masks,
                           tr_tags)
train_sampler = RandomSampler(train_data)
# Oluşturduğumuz tensor verisini modele vermek üzere dataloader değişkenine dönüştürüyoruz. 
train_dataloader = DataLoader(train_data,
                              sampler=train_sampler, 
                              batch_size=bs)

valid_data = TensorDataset(val_inputs,
                           val_masks, 
                           val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,
                              sampler=valid_sampler,
                              batch_size=bs)

In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW

In [ ]:
#Modelimizi oluşturuyoruz. Önceden tanımladığımız tokenizer ı 
#modeli fine tune etmek için da modele veriyoruz.
model = BertForTokenClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
# Modelimizin yapısı bu şekildedir.
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# Parametre optimizasyonu yapıyoruz.
FULL_FINETUNING = True
if FULL_FINETUNING:
    
    param_optimizer = list(model.named_parameters())
    
    no_decay = ['bias', 'gamma', 'beta']
    
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=4e-5,
    eps=1e-8
)
# Optimizer olarak Adam Optimizer kullanılıyor.

In [ ]:
# Toplam eğitim adım(epoch) sayısını ve kaç kere eğitim yapılacağı sayısını belirliyoruz.

from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

#  Öğrenmenin daha verimli olabilmesi ve learning rate optimizasyonu için bir scheduler yaratılıyor.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
# Modelin nasıl skorlanacağını tanımlıyoruz.
!pip install seqeval
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=3c1970e3537ab6062db9d1aefd2a0afb70e25162fc3a129e7b2b244da2e58e26
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
# Her epochtan sonra ortalama loss'u daha sonra çizdirebilmek için saklıyoruz.
loss_values, validation_loss_values = [], []
filename_iterator = 1
for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    #  Modelimizi eğitiyoruz.
    model.train()
    # Her epoch'taki 
    total_loss = 0

    # Eğitimin döngüsü
    for step, batch in enumerate(train_dataloader):
        
        # Partilere gpu ekle
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # Geriye doğru geçiş yapmadan önce hesaplanmış değerleri temizliyoruz.
        model.zero_grad()
        
        # forward pass
        # Etiketleri verdiğimizden dolayı bize model çıktısı olarak loss'u döndürür.
        outputs = model(b_input_ids, 
                        token_type_ids=None,
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # Kaybı alıyoruz.
        loss = outputs[0]
        
        
        # Gradient'leri hesaplamak için backward pass yapıyoruz.
        loss.backward()
        
        # Eğitimin loss'unu izliyoruz.
        total_loss += loss.item()
        
        
        # "Exploding gradients" problemini önlemek için gradient'lerin normunu kırpıyoruz.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
                                       max_norm=max_grad_norm)
        
        # Parametreleri update ediyoruz.
        optimizer.step()
        
        # Learning rate'i update ediyoruz.
        scheduler.step()

    # Eğitim verisi üzerindeki average loss'u hesaplıyoruz.
    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average train loss: {avg_train_loss}')

    # Öğrenme eğrisini çizdirebilmek için loss değerini saklıyoruz.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # Her eğitilen epochtan sonra doğrulama verisetinin performansına bakıyoruz.

    # Modeli değerlendirme moduna getiriyoruz.
    model.eval()
    
    # Her epochta doğrulama verisetinin loss'unu sıfırlıyoruz.
    eval_loss, eval_accuracy = 0, 0
    
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Bellek tasarrufu ve doğrulamayı hızlandırmak için 
        # Modele gradientleri hesaplamamasını veya depolamamasını söylüyoruz.
        with torch.no_grad():
            
            # Logit tahminlerini hesaplıyoruz.
            # Labelları sağlamadığımız için bu losslar yerine logit tahminlerini döndürüyoruz.
            outputs = model(b_input_ids, 
                            token_type_ids=None,
                            attention_mask=b_input_mask, 
                            labels=b_labels)
        
        # Logit ve labelları CPU'ya taşıyoruz.
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Test cümlesi için accuracy hesaplıyoruz.
        eval_loss += outputs[0].mean().item()
        eval_accuracy += flat_accuracy(logits, label_ids)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    validation_loss_values.append(eval_loss)
    print(f'Validation loss: {eval_loss}')
    print(f'Validation Accuracy: {eval_accuracy / nb_eval_steps}')
    
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print(f'Validation F1-Score: {f1_score(pred_tags, valid_tags)}')
    torch.save(model.state_dict(), "torch_bert_epoch_"+str(filename_iterator)+".pt")
    filename_iterator += 1
    print("Model is saved")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Average train loss: 0.37209480845184356
Validation loss: 0.3224814834940659
Validation Accuracy: 0.2919133339900182
Validation F1-Score: 0.636936273328698


Epoch:  33%|███▎      | 1/3 [1:55:52<3:51:44, 6952.50s/it]

Model is saved
Average train loss: 0.28993702177539443
Validation loss: 0.31200241094045605
Validation Accuracy: 0.2939714787609291
Validation F1-Score: 0.6684371242082955


Epoch:  67%|██████▋   | 2/3 [3:51:46<1:55:52, 6952.82s/it]

Model is saved
Average train loss: 0.23914640770463522
Validation loss: 0.3173462623206295
Validation Accuracy: 0.29459558754549947
Validation F1-Score: 0.6755935169768004


Epoch: 100%|██████████| 3/3 [5:47:36<00:00, 6952.14s/it]

Model is saved
